# Cas d'usage n°1

Après récupération d'un jeu de données quelconque (ici les industries classées pour la protection de l'environnement = ICPE de la région Hauts-de-France), l'objectif est de retrouver les codes communes manquants.

## Import des modules

In [1]:
import os
import pandas as pd
from requests_cache import CachedSession
from tqdm import tqdm

from french_cities import find_city

## Constitution du jeu de données : récupération des ICPE de la région Hauts-de-France

In [2]:
code_region = "32"
page_size = 1000

s = CachedSession()
r = s.get(
    "https://georisques.gouv.fr/api/v1/installations_classees",
    params={"page": "1", "page_size": page_size, "region": code_region},
)
number_pages = r.json()["total_pages"]
for x in tqdm(range(number_pages), desc="querying georisques", leave=False):
    try:
        data
    except NameError:
        data = []
    else:
        r = s.get(
            "https://georisques.gouv.fr/api/v1/installations_classees",
            params={
                "page": x + 1,
                "page_size": page_size,
                "region": code_region,
            },
        )
    finally:
        data += r.json()["data"]
    if not r.json()["next"]:
        break
data = pd.DataFrame(data)

## Analyse du jeu de données obtenu

In [3]:
data

,raisonSociale,adresse1,codePostal,codeInsee,commune,longitude,latitude,bovins,porcs,volailles,...,serviceAIOT,regime,rubriques,inspections,documentsHorsInspection,date_maj,adresse2,codeNaf,siret,adresse3
0,SCEA FERME LABALETTE,Lieu-dit Le Dièvre,62860,62739,Sains-lès-Marquion,3.083774,50.176359,False,False,False,...,DREAL HdF,Enregistrement,"[{'numeroRubrique': '2760', 'nature': 'Install...","[{'dateInspection': '2024-06-28', 'fichierInsp...",[{'identifiantFichier': 'ZRFGgSoRBn1aqafaBiI4a...,2024-09-25/05-30-04,NaN,NaN,NaN,NaN
1,ISDI de Saint-Laurent-Blangy,Zone des Trois Fontaines,62223,62753,Saint-Laurent-Blangy,2.809392,50.303387,False,False,False,...,DREAL HdF,Non ICPE,[],[{'dateInspection': '2019-02-04'}],[],2024-05-30/10-47-45,rue Henri Becquerel,NaN,NaN,NaN
2,RECYCL'ELECTRONIC SARL,2 Bis Rue <charles Duquesnoy,62270,62361,Frévent,2.293696,50.267633,False,False,False,...,DREAL HdF,Autres régimes,[],[{'dateInspection': '2015-09-29'}],[],2024-05-15/17-11-35,NaN,38,53748769600016,NaN
3,ISDI -TCPA (PATINIER A),rue de la Gare,62470,62197,Camblain-Châtelain,2.429339,50.478130,False,False,False,...,DREAL HdF,Non ICPE,[],[{'dateInspection': '2016-12-06'}],[],2024-05-30/10-47-45,Hâmeau de La Ferté au lieu dit Le Petit Pingue...,NaN,33255491400025,NaN
4,LAV'ALIM,Zone Industrielle Le Royeux,02430,02340,Gauchy,3.293073,49.822421,False,False,False,...,DREAL HdF,Autres régimes,[],[{'dateInspection': '2020-02-24'}],[],2023-11-29/11-19-15,5 Avenue de l'Europe,81,48908926800036,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12360,LIDL,1 B RUE DE CALAIS,59820,59273,GRAVELINES,2.118673,50.986002,False,False,False,...,DREAL HdF,Non ICPE,[],[{'dateInspection': '2024-12-02'}],[],2024-12-27/11-25-46,NaN,47,34326262204620,NaN
12361,NORAUTO,ZAL n°1 - Rue Bernard Chochoy,62800,62510,Liévin,2.773560,50.422542,False,False,False,...,DREAL HdF,Non ICPE,[],[{'dateInspection': '2024-12-16'}],[],2024-12-18/15-46-11,NaN,NaN,48047015200418,NaN
12362,CAPART Julien,Parcelle cadastrée ZN 4,80340,80432,Herleville,2.739324,49.874223,False,False,False,...,DREAL HdF,Non ICPE,[],[{'dateInspection': '2024-12-09'}],[],2024-12-20/11-16-15,NaN,NaN,NaN,NaN
12363,GCA SUPPLY PACKING,78 RUE HENRI BARBUSSE,59224,59589,THIANT,3.442195,50.307796,False,False,False,...,DREAL HdF,Non ICPE,[],[{'dateInspection': '2024-12-10'}],[],2024-12-23/11-45-20,NaN,16,35253392100909,NaN


## ICPE dépourvues de codes commune INSEE

In [4]:
data.codeInsee.isnull().value_counts().to_frame("Nombre de codes manquants :")

,Nombre de codes manquants :
codeInsee,
False,12112
True,253


##### A date du 14/02/2025, le jeu de données est constitué de 12365 lignes et 31 colonnes, dont 253 codes communes INSEE manquants. Pourtant, d'autres champs
sont disponibles et exploitables pour retrouver les communes manquantes :
* des champs adresse ;
* un libellé de commune ;
* des coordonnées géographiques et référentiel de projection ;
* un code postal.

## Complétion des données avec `french-cities`

### Extraction des données manquantes

In [5]:
missing = data[data.codeInsee.isnull()].copy()

Au besoin, vérifier que le système de projection des coordonnées est en EPSG 2154 (lorsqu'il est connu) :

In [6]:
missing[["systemeCoordonneesAIOT"]].fillna("est manquant").value_counts().to_frame()

,count
systemeCoordonneesAIOT,
est manquant,236
2154,17


### Concaténation des adresses :

In [7]:
cols = [f"adresse{x}" for x in range(1, 4)]
addresses = (
    missing[cols[0]]
    .str.cat(missing[cols[1:]], sep=" ", na_rep="")
    .str.replace(" +", " ", regex=True)
    .str.strip(" ")
)
missing["adresse"] = addresses

### Recherche des communes manquantes à l'aide de `french-cities` :

In [8]:
filled = find_city(
    missing,
    year="last",
    x="coordonneeXAIOT",
    y="coordonneeYAIOT",
    epsg=2154,
    city="commune",
    dep=False,
    address="adresse",
    postcode="codePostal",
    use_nominatim_backend=False,
    field_output="newCodeInsee",
)
filled

No data found !area for ultra-marine territories:   0%|                                                                              | 0/8 [00:00<?, ?it/s]
No data found !
No data found !
No data found !
No data found !area for ultra-marine territories:  50%|███████████████████████████████████                                   | 4/8 [00:02<00:02,  1.39it/s]
No data found !
                                                                                                                                                           

,raisonSociale,adresse1,codePostal,codeInsee,commune,longitude,latitude,bovins,porcs,volailles,...,rubriques,inspections,documentsHorsInspection,date_maj,adresse2,codeNaf,siret,adresse3,adresse,newCodeInsee
16,SARL FINANCIERE VARET,"Lieu-dit ""les Huit Cannes""",62118,NaN,Fampoux,-2.279526,45.502618,False,False,False,...,"[{'numeroRubrique': '2760', 'nature': 'Install...","[{'dateInspection': '2024-06-25', 'fichierInsp...",[{'identifiantFichier': 'xwOI57mkibOLwaTaqUaWl...,2024-09-21/05-30-29,NaN,84,37963809100023,NaN,"Lieu-dit ""les Huit Cannes""",62323
502,GSMC Market,ZAC DU PONT SANS PAREIL,62610,NaN,BOIS EN ARDRES,NaN,NaN,False,False,False,...,[],[{'dateInspection': '2017-11-10'}],[],2024-06-12/12-42-23,NaN,NaN,75202140200025,NaN,ZAC DU PONT SANS PAREIL,62038
2782,VIOLET Bernard,La Grande Cote,60180,NaN,Nogent-sur-Oise,114.936723,-89.574265,False,False,False,...,"[{'numeroRubrique': '2510', 'nature': 'Carrièr...","[{'dateInspection': '2024-07-25', 'fichierInsp...",[{'identifiantFichier': '87af46b17c164f4abec88...,2024-10-21/18-18-00,NaN,08,69637059200029,NaN,La Grande Cote,60463
3675,ALTEA MSO,ZAC du Plateau,02200,NaN,Ploisy,NaN,NaN,False,False,False,...,[],[],[{'identifiantFichier': '10a4a1f0d48c4e9792982...,1900-04-25/00-09-21,NaN,NaN,74572170400039,NaN,ZAC du Plateau,02607
3751,LES FRUITS ROUGES AISNE SA,1 rue Jean Bodin,02000,NaN,Laon,-1.363081,-5.983856,False,False,False,...,"[{'numeroRubrique': '1511', 'nature': 'Entrepô...","[{'dateInspection': '2022-09-21', 'fichierInsp...",[{'identifiantFichier': '8abb00ab7c503f14017c5...,2024-08-21/15-45-33,NaN,46,37897020600025,NaN,1 rue Jean Bodin,02408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12148,CRAPET Jean-Louis,le fond du Marquoi,62550,NaN,Bailleul-lès-Pernes,-5.525510,42.559454,False,False,False,...,[],"[{'dateInspection': '2023-10-06', 'fichierInsp...",[],2023-11-11/02-33-09,Chemin de la Drève,NaN,88407344600012,NaN,le fond du Marquoi Chemin de la Drève,62071
12166,PRESSING DES PORTES DE L'AVESNOIS,RTE DE VALENCIENNES,59530,NaN,LE QUESNOY,-0.001143,0.001046,False,False,False,...,[],"[{'dateInspection': '2023-10-11', 'fichierInsp...",[],2024-06-24/12-59-07,CC LES PORTES DE L'AVESNOIS ZAE OUEST,96,85053052800015,NaN,RTE DE VALENCIENNES CC LES PORTES DE L'AVESNOI...,59481
12167,VAL PRESS,72 AV DE DENAIN,59300,NaN,VALENCIENNES,-0.001105,0.001474,False,False,False,...,[],"[{'dateInspection': '2023-10-11', 'fichierInsp...",[],2024-06-21/13-20-25,NaN,96,43404025900013,NaN,72 AV DE DENAIN,59606
12212,NORBLOC,50 route de Tournai,59226,NaN,Lecelles,-0.001220,-0.001001,False,False,False,...,[],"[{'dateInspection': '2024-02-16', 'fichierInsp...",[],2024-06-12/03-32-24,NaN,NaN,89528842100012,NaN,50 route de Tournai,59335


### Réinjection les codes manquants dans le dataframe complet :

In [9]:
data = data.join(filled[["newCodeInsee"]])
data["codeInsee"] = data["codeInsee"].combine_first(data["newCodeInsee"])
data = data.drop("newCodeInsee", axis=1)

### Evaluation des résultats

In [10]:
data.codeInsee.isnull().value_counts().to_frame("Nombre de codes manquants :")

,Nombre de codes manquants :
codeInsee,
False,12364
True,1


### Données toujours manquantes

In [11]:
data[data.codeInsee.isnull()]

,raisonSociale,adresse1,codePostal,codeInsee,commune,longitude,latitude,bovins,porcs,volailles,...,serviceAIOT,regime,rubriques,inspections,documentsHorsInspection,date_maj,adresse2,codeNaf,siret,adresse3
8500,ATELIERS DES EPICES ET CONDIMENTS,19 RUE DE LA GARE,62360,NaN,PONT DE BRIQUES,NaN,NaN,False,False,False,...,DREAL HdF,Non ICPE,[],[{'dateInspection': '2015-01-12'}],[],2024-12-24/15-00-00,SAINT ETIENNE AU MONT,NaN,53197729600023,NaN


A date du 14/02/2025, une seule commune n'a pas été trouvée.

Effectivement, dans ce cas de figure, le lieu-dit (PONT DE BRIQUES) et la commune (SAINT ETIENNE AU MONT) ont été inversés : ceci explique que le score de la base adresse nationale n'ait pas été jugé suffisamment bon pour que le résultat de Saint-Etienne-au-Mont puisse être retenu...

Si cette fois, on décide d'utiliser l'API Nominatim en dernier recours, le code devient :

In [12]:
# On isole la(es) ligne(s) manquante(s)
missing = data[data.codeInsee.isnull()].copy()

# On concatène de nouveau les champs adresses :
cols = [f"adresse{x}" for x in range(1, 4)]
addresses = (
    missing[cols[0]]
    .str.cat(missing[cols[1:]], sep=" ", na_rep="")
    .str.replace(" +", " ", regex=True)
    .str.strip(" ")
)
missing.loc[:, "adresse"] = addresses

# Et on spécifie l'usage de Nominatim
missing = find_city(
    missing,
    year="last",
    x=False,
    y=False,
    dep=False,
    epsg=2154,
    city="commune",
    address="adresse",
    postcode="codePostal",
    use_nominatim_backend=True,
    field_output="newCodeInsee",
)

Utilisons pynsee pour récupérer la liste des communes pour contrôler le résultat (et pas simplement son code commune) :

In [13]:
# Présentation des résultats
from pynsee.localdata import get_area_list
cities = get_area_list("communes", date="*")
missing["newCodeInsee"].to_frame().merge(cities, left_on="newCodeInsee", right_on="CODE")

,newCodeInsee,CODE,URI,AREA_TYPE,DATE_CREATION,TITLE_SHORT,DETERMINER_TYPE,TITLE,DATE_DELETION
0,62746,62746,http://id.insee.fr/geo/commune/adc106e4-5055-4...,Commune,1943-01-01,Saint-Étienne-au-Mont,0,Saint-Étienne-au-Mont,None


💡 Nota : l'exécution de Nominatim ne conduit pas systématiquement au même résultat (et même parfois ne produit pas de résultat). Cela n'est pas totalement absurde, le hameau manquant étant à cheval sur plusieurs communes. Les résultats fournis restent  généralement pertinents.